In [1]:
# Displays or modifies the intervals between automatic Notebook saves. The default setting is every 120 seconds.
%autosave 60
# Load all the import for every execution of a script.
%load_ext autoreload
%autoreload 2

Autosaving every 60 seconds


# Import

In [2]:
import harmonize as hz
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from shapely.geometry import mapping

If issue with cfgrib -->  pip install eccodes>=2.21.0

# Load data sets

In [3]:
# Create a path to the data directory
path_data = "../data/Raw/"

In [4]:
# Load the data set
ndvi = xr.open_dataset(path_data +'Raw_NDVI_16D_1km.nc')
lai = xr.open_dataset(path_data +'Raw_LAI_8D_500m.nc')
evap = xr.open_dataset(path_data +'Raw_Evap_8D_500m.nc')
era = xr.open_dataset(path_data +'Raw_weather_4H_9km.nc')
lst_night = xr.open_dataset(path_data +'Raw_LST_Night_1D_1km.nc')
lst_day = xr.open_dataset(path_data +'Raw_LST_Day_1D_1km.nc')
active_fire = xr.open_dataset(path_data +'Raw_ActiveFire_500m.nc')
burn_mask =xr.open_dataset(path_data +'Raw_BurnMask_1km.nc')
fwi = xr.open_mfdataset(path_data+'/Raw_Fwi/*.nc', combine='by_coords', chunks=None)
density = rxr.open_rasterio(path_data +'fra_pd_2015_1km_UNadj.tif', masked=True).squeeze()

# Pre-process the datasets

## Variables selection

Variables of interest:
- Fpar_500m: Fraction of Photosynthetically Active Radiation (continuous) min: 0 max: 2.54 unit: percentage
- _1_km_16_days_EVI: Ehanced Vegetation Index (continuous) min: -02 max: 0.8983 unit: none
- ET_500m: Total Evapotranspiration (continuous) min: 0 max: 3.277e+03 unit: kg/m²/8day
- u10: Eastward component of the 10m wind (continuous) min: -17.09 max: 15.15 unit: m/s
- v10: Northward component of the 10m wind (continuous) min: -15.89 max: 14.17 unit: m/s
- t2m: Temperature of air at 2m above the surface of land, sea or in-land waters. The temperature measured in kelvin can be converted to degrees Celsius (°C) by subtracting 273.15. (continuous) min: 244 max: 317 unit: K
- tp: precipitation (continuous) min: 0 max: 0.122 unit: m
- LST_Day_1km: Land Surface Temperature day (continuous) min: 240.0 max: 332.9 unit: K
- LST_Night_1km:**Land Surface Temperature day (continuous) min: 227.3 max: 321.2 unit: K
- fwi-daily-proj: Fire Weather Index (continuous) min: 1 max: 473 unit: dimensionless
- First_Day: First day of the year of reliable change detection (categorical) min: 0 max: 366 unit: day
- Last_Day: Last day of the year of reliable change detection (categorical) min: 0 max: 366 unit: day
- FireMask

In [5]:
# Select the variables of interest
ndvi_filter = ndvi['_1_km_16_days_EVI']
lai_filter = lai['Fpar_500m']
evap_filter = evap['ET_500m']
era_filter = era[['u10', 'v10', 't2m', 'tp']]
lst_night_filter = lst_night['LST_Night_1km']
lst_day_filter = lst_day['LST_Day_1km']
fwi_filter = fwi['fwi-daily-proj']
active_fire_filter = active_fire[['First_Day', 'Last_Day']]
burn_mask_filter = burn_mask['FireMask']

## Setting the CRS of the datasets

In [10]:
era.filter

AttributeError: 'Dataset' object has no attribute 'filter'

In [8]:
# rename dimesion of era5
era_filter = era_filter.rename_dims({'x': 'xdim', 'y': 'ydim'})


ValueError: cannot rename 'x' because it is not a dimension in this dataset

In [6]:
# Swap dimensions of era5
era_filter = era_filter.swap_dims({'x': 'xdim', 'y': 'ydim'})

ValueError: cannot swap from dimension 'x' because it is not an existing dimension

In [11]:
# Create a dictionary with the CRS of the datasets
crs = {'ndvi': ndvi_filter.rio.crs,
       'lai': lai_filter.rio.crs,
       'evap': evap_filter.rio.crs,
       'era': era_filter.rio.crs,
       'lst_night': lst_night_filter.rio.crs,
       'lst_day': lst_day_filter.rio.crs,
       'fwi': fwi_filter.rio.crs,
       'active_fire': active_fire_filter.rio.crs,
       'burn_mask': burn_mask_filter.rio.crs,
       'density': density.rio.crs}



## Setting the extent of the datasets

In [12]:
# Create a CRS object from a poj4 string for sinuoidal projection
crs_sinu = rasterio.crs.CRS.from_string("+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")

In [13]:
# Define a crs for rotated pole
crs_rotated_pole = rasterio.crs.CRS.from_string("+proj=ob_tran +o_proj=longlat +o_lon_p=-162 +o_lat_p=39.25 +lon_0=180 +to_meter=0.01745329")

In [14]:
# Set the CRS of the data sets with hz.define_crs()
ndvi_filter = hz.define_crs(ndvi_filter, crs_sinu)
lai_filter = hz.define_crs(lai_filter, crs_sinu)
evap_filter = hz.define_crs(evap_filter, crs_sinu)
era_filter = hz.define_crs(era_filter, 4326)
lst_night_filter = hz.define_crs(lst_night_filter, crs_sinu)
lst_day_filter = hz.define_crs(lst_day_filter, crs_sinu)
fwi_filter = hz.define_crs(fwi_filter, crs_rotated_pole)
active_fire_filter = hz.define_crs(active_fire_filter, crs_sinu)
burn_mask_filter = hz.define_crs(burn_mask_filter, crs_sinu)
density = hz.define_crs(density, 4326)

In [15]:
# Define the AOI
aoi = hz.define_area_of_interest(path_data+'AreaOfInterest.zip')

In [16]:
# Clip the data sets to the AOI
era_filter = hz.clip_to_aoi(era_filter, aoi)
density = hz.clip_to_aoi(density, aoi)

In [17]:
f#wi_filter = hz.clip_to_aoi(fwi_filter, aoi)
# Error: "NoDataInBounds: No data found in bounds. Data variable: fwi-daily-proj

NameError: name 'f' is not defined

In [18]:
# Projection of fwidailyproj to EPSG 4326
#fwi_filter = fwi_filter.rio.reproject('EPSG:4326')

## Projection and regridding of the datasets

In [19]:
# create a common grid
common_grid = rxr.open_rasterio(path_data +'Raw_LST_Day_1D_1km.nc').isel(time=0)

In [20]:
# Interpolate density to common grid
density_proj = density.rio.reproject_match(common_grid, resampling=rasterio.enums.Resampling.bilinear)

In [21]:
# Interpolate lai to common grid
lai_proj = lai_filter.rio.reproject_match(common_grid, resampling=rasterio.enums.Resampling.bilinear)

In [22]:
# Interpolate evap to common grid
evap_proj = evap_filter.rio.reproject_match(common_grid, resampling=rasterio.enums.Resampling.bilinear)

In [23]:
# Different method to interpolate the active fire data set
active_fire_filter_proj = active_fire_filter.interp(ydim=ndvi["ydim"], xdim=ndvi['xdim'])

In [24]:
print(density_proj.name)

None


In [ ]:
# rename dimension of density
density_proj = density_proj.rename_dims({'latitude': 'ydim', 'longitude': 'xdim'})

In [34]:
density_proj.name = 'density'

In [21]:
# Print the name of all the data set
print(density_proj.name)
print(lai_proj.name)
print(evap_proj.name)
print(ndvi_filter.name)
print(lst_night_filter.name)
print(lst_day_filter.name)
print(era_filter.name)
print(burn_mask_filter.name)



None
Fpar_500m
ET_500m
_1_km_16_days_EVI
LST_Night_1km
LST_Day_1km


AttributeError: 'Dataset' object has no attribute 'name'

#### Interpolate the era data set to the same grid as the lst_day_filter data set

In [25]:
# Project era to sinuoidal projection
era_sinu = era_filter.rio.reproject(crs_sinu)

In [26]:
era_sinu

<xarray.Dataset>
Dimensions:      (x: 30, y: 24, time: 22975)
Coordinates:
  * x            (x) float64 3.385e+05 3.478e+05 3.57e+05 ... 5.977e+05 6.07e+05
  * y            (y) float64 5.005e+06 4.995e+06 ... 4.801e+06 4.792e+06
  * time         (time) datetime64[ns] 2010-01-01T04:00:00 ... 2022-07-31T20:...
    spatial_ref  int32 0
Data variables:
    u10          (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    v10          (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    t2m          (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    tp           (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [30]:
# change dimension name of era sinu
era_sinu = era_sinu.rename_dims({'y': 'ydim', 'x': 'xdim'})

ValueError: cannot rename 'y' because it is not a dimension in this dataset

In [31]:
density_proj

<xarray.DataArray (y: 253, x: 298)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * x            (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y            (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
    band         int32 1
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  53646.03515625
    STATISTICS_MEAN:     119.46772920052
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   644.66432858711
    scale_factor:        1.0
    add_offset:          0.0

In [35]:
# change density_proj to xarray dataset
density_proj = density_proj.to_dataset()

In [36]:
#Change name of the dimension of the density data set
density_proj = density_proj.rename_dims({'y': 'ydim', 'x': 'xdim'})

In [37]:
density_proj

<xarray.Dataset>
Dimensions:      (xdim: 298, ydim: 253)
Coordinates:
  * x            (xdim) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y            (ydim) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
    band         int32 1
    spatial_ref  int32 0
Dimensions without coordinates: xdim, ydim
Data variables:
    density      (ydim, xdim) float32 nan nan nan nan nan ... nan nan nan nan

In [29]:
era_sinu

<xarray.Dataset>
Dimensions:      (x: 30, xdim: 24, time: 22975)
Coordinates:
  * x            (x) float64 3.385e+05 3.478e+05 3.57e+05 ... 5.977e+05 6.07e+05
  * y            (xdim) float64 5.005e+06 4.995e+06 ... 4.801e+06 4.792e+06
  * time         (time) datetime64[ns] 2010-01-01T04:00:00 ... 2022-07-31T20:...
    spatial_ref  int32 0
Dimensions without coordinates: xdim
Data variables:
    u10          (time, xdim, x) float32 nan nan nan nan nan ... nan nan nan nan
    v10          (time, xdim, x) float32 nan nan nan nan nan ... nan nan nan nan
    t2m          (time, xdim, x) float32 nan nan nan nan nan ... nan nan nan nan
    tp           (time, xdim, x) float32 nan nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [40]:
# Interpolate era_sinu to common grid
#era_sinu_proj = era_sinu.rio.reproject_match(common_grid, resampling=rasterio.enums.Resampling.bilinear)

MemoryError: Unable to allocate 6.45 GiB for an array with shape (22975, 253, 298) and data type float32

The dataset is too big, we will create a subset of the data set and reproject separately the data set to the same grid.

In [43]:
era_1 = era_sinu['v10']
era_2 = era_sinu['t2m']
era_3 = era_sinu['u10']
era_4 = era_sinu['tp' ]

In [44]:
# Interpolate era_1 to the same grid as lst_day_filter
era_1_interp = hz.interpolate_to_common_grid(era_1, common_grid)

MemoryError: Unable to allocate 6.45 GiB for an array with shape (22975, 253, 298) and data type float32

In [ ]:
# Interpolate era_2 to the same grid as lst_day_filter
era_2_interp = hz.interpolate_to_common_grid(era_2, common_grid)

In [ ]:
# Interpolate era_3 to the same grid as lst_day_filter
era_3_interp = hz.interpolate_to_common_grid(era_3, common_grid)

In [ ]:
# Interpolate era_4 to the same grid as lst_day_filter
era_4_interp = hz.interpolate_to_common_grid(era_4, common_grid)

Still too big, we will first downsampling to a daily resolution and then reproject to the same grid.

In [45]:
# Downsample the era data set to a daily resolution
era_sinu_daily = hz.resample_to_daily(era_sinu)

In [46]:
# Interpolate era_sinu_daily to common grid
era_sinu_daily_proj = era_sinu_daily.rio.reproject_match(common_grid, resampling=rasterio.enums.Resampling.bilinear)

MemoryError: Unable to allocate 2.58 GiB for an array with shape (4595, 253, 298) and data type float64